In [1]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import math
%matplotlib inline

import sys
sys.path.insert(0, '../../scripts')

import stft_zoom, display, detect_musical_regions
from util import *
import mappings
import pickle
import PIL
import IPython.display
from classes import SingleResSpectrogram, MultiResSpectrogram
import glob
from aug_density_map import *
from mappings import *

import timeit
import csv
import mido

/Library/Python/3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/Library/Python/3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
#do mauricio
sys.path.insert(0, '../../scripts/mauricio_solutions/')
import lukin_todd, swgm, local_sparsity, util_m

In [3]:
from IPython.display import Audio
sound_file = '/Users/nicolas/Downloads/camera.mp3'

### A comparison of computational cost between different TFP representations

#### STFT, CQT, Lukin & Todd, SWGM, SLS x our multiresolution spectrogram

The objective is to evaluate the computational cost (execution CPU time and memory used) for a given max resolution of the representation, and to obtain a cost curve for these different representations.

Parameters of this experiment are:

    - max freq. resolution:  the cost of 86.13, 43.06, 21.53, 10.77, 5.38, 2.69 and 1.34 Hz per bin (the equivalent of using windows of [512, 1024, 2048, 4096, 8192, 16384, 32768] on a 44100 Hz sampled signal) will be evaluated
    - files per data-point: 10 files?
    - CQT parameters: fmin = 20, 10 octaves, resolution to match cents between 100-200Hz
    - Lukin & Todd kernel size same as Mauricio kernel size 
    - Our subregion size: test several
    - Normalization simplified

In [4]:
def hz_to_cents(hz_resolution):
    # Converts from Hz res to cent resolution in the octave from 100-200Hz
    delta_f = 100
    cent_delta_f = delta_f/1200
    return hz_resolution / cent_delta_f

def hz_to_binperoct(hz_resolution):
    return int(np.ceil(1200 / hz_to_cents(hz_resolution)))

In [12]:
def calc_kernel_size(window_lengths, energy=False):
    if energy:
        P = 2
    else:
        P = 5
    f_size = P * window_lengths[-1]/window_lengths[0]
    if not f_size % 2 > 0:
        f_size += 1
    
    return [int(f_size), 5]

def LT(y, window_lengths, kernel):
    specs = util_m.get_spectrograms(y, windows=window_lengths)
    specs = util_m.interpol_and_normalize(specs)
    return lukin_todd.lukin_todd(specs, kernel)

def swgm_time(y, window_lengths):
    specs = util_m.get_spectrograms(y, windows=window_lengths)
    specs = util_m.interpol_and_normalize(specs)
    return swgm.SWGM(specs)

def SLS_time(y, window_lengths, kernel_anal, kernel_energy):
    specs = util_m.get_spectrograms(y, windows=window_lengths)
    specs = util_m.interpol_and_normalize(specs)
    return local_sparsity.smoothed_local_sparsity(specs, kernel_anal, kernel_energy)

def our_solution(y, res, kernel, model, pct, sr=44100, n_fft=512, hop_size=128):
    spec = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_size))
    time_span = [0,len(y)/sr]
    x_axis, y_axis = stft_zoom.get_axes_values(sr, 0, time_span, spec.shape) 
    base_spec = SingleResSpectrogram(spec, x_axis, y_axis)
    multires_spec = MultiResSpectrogram(base_spec)
    indices, original_shape = detect_musical_regions.detect_musical_regions(model, spec, kernel=kernel, mode='pct', pct_or_threshold=pct, n_fft=n_fft, hop_size=hop_size)
    to_be_refined = detect_musical_regions.musical_regions_to_ranges(indices, original_shape, x_axis, y_axis, kernel, n_fft=n_fft, hop_size=hop_size)

    stft_zoom.set_signal_bank(y,kernel)

    for subregion in to_be_refined:
        freq_range = subregion[0]
        time_range = subregion[1]
        spec_zoom, x_axis, y_axis, new_sr, window_size, hop_size = stft_zoom.stft_zoom(y, freq_range, time_range, sr=sr, original_window_size=n_fft, k=res)
        refined_subspec = SingleResSpectrogram(spec_zoom, x_axis, y_axis)
        multires_spec.insert_zoom(multires_spec.base_spec, refined_subspec, zoom_level=1)
        
    return multires_spec

def our_solution_multilevel(y, res, kernel_list, model, pct_list, sr=44100, n_fft=512, hop_size=128):
    o_n_fft = n_fft
    o_hop_size = hop_size
    spec = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_size))
    time_span = [0,len(y)/sr]
    x_axis, y_axis = stft_zoom.get_axes_values(sr, 0, time_span, spec.shape) 
    base_spec = SingleResSpectrogram(spec, x_axis, y_axis)
    multires_spec = MultiResSpectrogram(base_spec)
    
    kernel = kernel_list[0]
    indices, original_shape = detect_musical_regions.detect_musical_regions(model, spec, kernel=kernel, mode='pct', pct_or_threshold=pct_list[0], n_fft=n_fft, hop_size=hop_size)
    to_be_refined = detect_musical_regions.musical_regions_to_ranges(indices, original_shape, x_axis, y_axis, kernel, n_fft=n_fft, hop_size=hop_size)

    stft_zoom.set_signal_bank(y,kernel)
    
    print(kernel, len(to_be_refined))
    for subregion in to_be_refined:
        freq_range = subregion[0]
        time_range = subregion[1]
        spec_zoom, x_axis, y_axis, new_sr, window_size, hop_size = stft_zoom.stft_zoom(y, freq_range, time_range, sr=sr, original_window_size=n_fft, k=res)
        refined_subspec = SingleResSpectrogram(spec_zoom, x_axis, y_axis, n_fft=window_size, hop_size=hop_size, sr=new_sr)
        multires_spec.insert_zoom(multires_spec.base_spec, refined_subspec, zoom_level=1)
        
    i = 1
    k = res
    for kernel in kernel_list[1:]:
        i += 1
        if i == 2:
            spec_list = multires_spec.first_zoom
        elif i == 3:
            spec_list = multires_spec.second_zoom
        elif i == 4:
            spec_list = multires_spec.third_zoom

        to_be_further_refined = []
        for spec_zoom in spec_list:
            spec        = np.array(spec_zoom.spec, dtype=float)
            x_axis      = spec_zoom.x_axis
            y_axis      = spec_zoom.y_axis
            sr          = spec_zoom.sr
            window_size = spec_zoom.n_fft
            hop_size    = spec_zoom.hop_size

            indices, original_shape = detect_musical_regions.detect_musical_regions(model_800, spec, mode='pct', pct_or_threshold=pct_list[i-1], kernel=kernel, n_fft=window_size, hop_size=hop_size, sr=sr, y_axis=y_axis)
            to_be_further_refined = to_be_further_refined + detect_musical_regions.musical_regions_to_ranges(indices, original_shape, x_axis, y_axis, kernel, sr=sr, hop_size=hop_size)

        k = k*2
        sr = 44100
        hop_size = o_hop_size
        n_fft = o_n_fft
        print(kernel, len(to_be_further_refined))
        for subregion in to_be_further_refined:
            freq_range = subregion[0]
            time_range = subregion[1]
            spec_zoom, x_axis, y_axis, new_sr, window_size, hop_size = stft_zoom.stft_zoom(y, freq_range, time_range, sr=sr, original_window_size=n_fft, k=k)
            refined_subspec = SingleResSpectrogram(spec_zoom, x_axis, y_axis, n_fft=window_size, hop_size=hop_size, sr=new_sr)
            multires_spec.insert_zoom(multires_spec.base_spec, refined_subspec, zoom_level=i, normalize=False)
    return multires_spec

In [6]:
def get_refine_map(midi_file, kernel, timestamp, num_time_frames, n_fft=512, hop_size=512//4):
    midi = mido.MidiFile(midi_file)
    proll = midi_to_piano_roll_new(midi, timestamp)
    proll_den = calc_map_aug2(sound_event_to_tfp(proll, num_time_frames, n_fft=n_fft), kernel, type='avg', n_fft=n_fft, hop_size=hop_size)
    return np.array(proll_den > 0, dtype=int)

def get_pct_to_refine(midi_file, kernel, timestamp, n_fft=512, hop_size=512//4):
    num_time_frames = int(np.ceil(44100 * 30 / hop_size))
    refine_map = get_refine_map(midi_file, kernel, timestamp, num_time_frames, n_fft=n_fft, hop_size=hop_size)
    return 100*(np.sum(refine_map)/refine_map.size)

# fazer: overlap dos detectados com kernel grande e todos kernel pequeno: N
#        overlap dos detectados com kernel pequeno e detectados kernel grande: n
#        o numero que queremos é n/N

# no caso real: N continua o mesmo, overlap dos detectado com kernel pequeno e refinados kernel grande = n

def in_range_list(range, range_list):
    for r in range_list:
        if range[0][0] >= r[0][0] and range[0][1] <= r[0][1]:
            if range[1][0] >= r[1][0] and range[1][1] <= r[1][1]:
                return True
    return False

def all_overlapping_ranges(range_list_1, range_list_2):
    # returns ranges in list_1 that are contained inside the regions defined by range_list_2
    overlapping_ranges = []
    for range in range_list_1:
        if in_range_list(range, range_list_2):
            overlapping_ranges.append(range)
    return overlapping_ranges

def get_pct_to_refine_multilevel(midi_file, kernel_list, timestamp, n_fft=512, hop_size=512//4, y=None):
# 1. pegar kernel primeiro nivel, ver no ground truth as regioes relevantes, guardar os ranges.
# 2. pegar kernel segundo, ver no ground truth as regioes relevantes, guardar os ranges.
# 3. ver overlap entre passo 2 e 1 = n
# 4. pegar kernel segundo, pegar ranges de todas as subregioes definidas por ele
# 5. ver overlap entre passo 4 e 1 = N
# 6. calcular pct, guardar regiões do passo 3 como as novas subregioes refinadas.
    spec = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_size))
    x_axis, y_axis = stft_zoom.get_axes_values(44100, 0, [0.0, 30.0], spec.shape) 
    num_time_frames = int(np.ceil(44100 * 30 / hop_size))
    first_level = get_refine_map(midi_file, kernel_list[0], timestamp, num_time_frames, n_fft=n_fft, hop_size=hop_size)
    pct_list = [100*(np.sum(first_level)/first_level.size)]
    indices_first_level = np.where(first_level.flatten() > 0)[0]
    range_list_first = detect_musical_regions.musical_regions_to_ranges(indices_first_level, first_level.shape, x_axis, y_axis, kernel_list[0], n_fft=n_fft, hop_size=hop_size)
    for kernel in kernel_list[1:]:
        second_level = get_refine_map(midi_file, kernel, timestamp, num_time_frames, n_fft=n_fft, hop_size=hop_size)
        indices_second_level = np.where(second_level.flatten() > 0)[0]
        all_subregions_second_level = np.where(second_level.flatten() > -1)[0]
        all_ranges_second_level = detect_musical_regions.musical_regions_to_ranges(all_subregions_second_level, second_level.shape, x_axis, y_axis, kernel, n_fft=n_fft, hop_size=hop_size)
        range_list_second = detect_musical_regions.musical_regions_to_ranges(indices_second_level, second_level.shape, x_axis, y_axis, kernel, n_fft=n_fft, hop_size=hop_size)
        n = len(all_overlapping_ranges(range_list_second, range_list_first))
        N = len(all_overlapping_ranges(all_ranges_second_level, range_list_first))
        pct_list.append((n/N) * 100)
        range_list_first = range_list_second
    return pct_list

In [6]:
file_name = '../../data/MIDI-Unprocessed_XP_09_R1_2004_05_ORIG_MID--AUDIO_09_R1_2004_06_Track06_wav.wav'
y, sr = librosa.load(file_name, sr=44100)
y = y[:44100*30]

In [7]:
file_names = []
for year in ['2004','2006','2008','2009','2011','2013','2014','2015']:
    path = '/Volumes/HD-NICO/vaio-backup/Documents/ime/compmus/mestrado/maestro-v2.0.0/' + year + '/*.wav'
    for file in glob.glob(path):
        file_names.append(file[:-4])

In [7]:
res_hz = [86.13, 43.06, 21.53, 10.77, 5.38, 2.69, 1.34]
res_window = [512, 1024, 2048, 4096, 8192, 16384, 32768]
model_200 = pickle.load(open('../renyi_shannon_prollharm_200.sav', 'rb'))
model_500 = pickle.load(open('../renyi_shannon_prollharm_500.sav', 'rb'))
model_800 = pickle.load(open('../renyi_shannon_prollharm_800.sav', 'rb'))

In [8]:
midi = '../../data/MIDI-Unprocessed_XP_09_R1_2004_05_ORIG_MID--AUDIO_09_R1_2004_06_Track06_wav.midi'
y, sr = librosa.load('../../data/MIDI-Unprocessed_XP_09_R1_2004_05_ORIG_MID--AUDIO_09_R1_2004_06_Track06_wav.wav', sr=44100)
timestamp = int((len(y) / sr) // 2) # pega o meio da gravação (só analisaremos 30 segundos do meio da música)
y = y[sr*timestamp:sr*(timestamp+30)]

In [14]:
j = 0

file_name = '../../data/MIDI-Unprocessed_XP_09_R1_2004_05_ORIG_MID--AUDIO_09_R1_2004_06_Track06_wav'

# for file in file_names[:5]:
for file in [file_name]:
    y, sr = librosa.load(file + '.wav', sr=44100)
    timestamp = int((len(y) / sr) // 2) # pega o meio da gravação (só analisaremos 30 segundos do meio da música)
    y = y[sr*timestamp:sr*(timestamp+30)]
    print(j, file)
    j += 1
    for i in range(len(res_window)):
        #STFT
        res = res_window[i]
        print(res)
        result_stft = %timeit -n 3 -r 3 -o librosa.stft(y, n_fft=res)
        #CQT
        bpo = hz_to_binperoct(res_hz[i])
        result_cqt = %timeit -n 3 -r 3 -o librosa.cqt(y, sr=sr, bins_per_octave=bpo, fmin=20, n_bins=10*bpo)
        # LUKIN-TODD, SLS, SWGM
        max_window = res_window[i]
        window_lengths = [int(max_window/8), int(max_window/2), max_window]
    #     kernel_anal = calc_kernel_size(window_lengths)
    #     kernel_energy = calc_kernel_size(window_lengths, energy=True)
    #     result_LT = %timeit -n 1 -r 1 -o LT(y, window_lengths, kernel_anal)
        result_SWGM = %timeit -n 3 -r 3 -o swgm_time(y, window_lengths)
    #     result_SLS = %timeit -n 1 -r 1 -o SLS_time(y, window_lengths, kernel_anal, kernel_energy)
        # OUR SOLUTION
        midi = file + '.midi'
        
        res = np.max([int(res_window[i] // 512), 1])
        n_fft = 512
        hop_size = n_fft//4
        if res == 1:
            res += 1
        pct_200 = get_pct_to_refine(midi, [200,200], timestamp, n_fft=n_fft, hop_size=hop_size)
        pct_500 = get_pct_to_refine(midi, [500,500], timestamp, n_fft=n_fft, hop_size=hop_size)
        pct_800 = get_pct_to_refine(midi, [800,800], timestamp, n_fft=n_fft, hop_size=hop_size)
        
        print('Pcts single level: ', pct_200, pct_500, pct_800)
        print('n_fft: ', n_fft)
        result_OURS_200 = %timeit -n 1 -r 1 -o our_solution(y, res, [200,200], model_200, pct_200, n_fft=n_fft, hop_size=hop_size)
        result_OURS_500 = %timeit -n 1 -r 1 -o our_solution(y, res, [500,500], model_500, pct_500, n_fft=n_fft, hop_size=hop_size)
        result_OURS_800 = %timeit -n 1 -r 1 -o our_solution(y, res, [800,800], model_800, pct_800, n_fft=n_fft, hop_size=hop_size)
        
        n_fft_2 = np.max([res_window[i]//2, 128])
        n_fft_3 = np.max([res_window[i]//4, 128])
        n_fft_4 = np.max([res_window[i]//8, 128])
        
        pct_multilevel_2 = get_pct_to_refine_multilevel(midi, [[1600,1600], [800,800]], timestamp, n_fft=n_fft_2, hop_size=n_fft_2//4, y=y)
        pct_multilevel_3 = get_pct_to_refine_multilevel(midi, [[1600,1600], [800,800], [400,400]], timestamp, n_fft=n_fft_3, hop_size=n_fft_3//4, y=y)
        pct_multilevel_4 = get_pct_to_refine_multilevel(midi, [[1600,1600], [800,800], [400,400], [200,200]], timestamp, n_fft=n_fft_4, hop_size=n_fft_4//4, y=y)
        
        print('Pcts multilevel: ', pct_multilevel_2, pct_multilevel_3, pct_multilevel_4)
        print('n_ffts multilevel: ', n_fft_2, n_fft_3, n_fft_4)
        result_OURS_2lvl = %timeit -n 1 -r 1 -o our_solution_multilevel(y, 2, [[1600,1600], [800,800]], model_800, pct_multilevel_2, sr=44100, n_fft=n_fft_2, hop_size=n_fft_2//4)
        result_OURS_3lvl = %timeit -n 1 -r 1 -o our_solution_multilevel(y, 2, [[1600,1600], [800,800], [400,400]], model_800, pct_multilevel_3, sr=44100, n_fft=n_fft_3, hop_size=n_fft_3//4)
        result_OURS_4lvl = %timeit -n 1 -r 1 -o our_solution_multilevel(y, 2, [[1600,1600], [800,800], [400,400], [200,200]], model_800, pct_multilevel_4, sr=44100, n_fft=n_fft_4, hop_size=n_fft_4//4)

#         result_file = 'results_' + str(res_window[i]) + '.csv'

#         with open(result_file, 'a') as csvfile:
#             writer = csv.writer(csvfile, delimiter=';')
#             writer.writerow([file, 'STFT', str(result_stft)])
#             writer.writerow([file, 'CQT', str(result_cqt)])
# #             writer.writerow(['LT', str(result_LT)])
#             writer.writerow([file, 'SWGM', str(result_SWGM)])
# #             writer.writerow(['SLS', str(result_SLS)])
#             writer.writerow([file, 'economic 200', str(result_OURS_200)])
#             writer.writerow([file, 'economic 500', str(result_OURS_500)])
#             writer.writerow([file, 'economic 800', str(result_OURS_800)])

0 ../../data/MIDI-Unprocessed_XP_09_R1_2004_05_ORIG_MID--AUDIO_09_R1_2004_06_Track06_wav
512
37.8 ms ± 1.57 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)
576 ms ± 105 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)
121 ms ± 2.02 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)
Pcts single level:  10.508149568552252 19.50980392156863 24.54954954954955
n_fft:  512
1.87 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
976 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
757 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
Pcts multilevel:  [25.0, 82.43243243243244] [15.555555555555555, 64.0, 80.0] [15.555555555555555, 64.0, 80.0, 85.9375]
n_ffts multilevel:  256 128 128
[1600, 1600] 27
[800, 800] 57
555 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
[1600, 1600] 14
[800, 800] 32
[400, 400] 72
550 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
[1600, 1600] 14
[800, 800] 32
[400, 400] 72
[200, 200] 192
714 ms ± 

In [11]:
n_fft_2 = 8192
pct_multilevel_2 = get_pct_to_refine_multilevel(midi, [[1600,1600], [800,800]], timestamp, n_fft=n_fft_2, hop_size=n_fft_2//4, y=y)

In [13]:
a = our_solution_multilevel(y, 2, [[1600,1600], [800,800]], model_800, pct_multilevel_2, sr=44100, n_fft=n_fft_2, hop_size=n_fft_2//4)

[1600, 1600] 58
[800, 800] 58


In [29]:
a.first_zoom[0].y_axis

array([221.85276074, 224.55828221, 227.26380368, 229.96932515,
       232.67484663, 235.3803681 , 238.08588957, 240.79141104,
       243.49693252, 246.20245399, 248.90797546, 251.61349693,
       254.3190184 , 257.02453988, 259.73006135, 262.43558282,
       265.14110429, 267.84662577, 270.55214724, 273.25766871,
       275.96319018, 278.66871166, 281.37423313, 284.0797546 ,
       286.78527607, 289.49079755, 292.19631902, 294.90184049,
       297.60736196, 300.31288344, 303.01840491, 305.72392638,
       308.42944785, 311.13496933, 313.8404908 , 316.54601227,
       319.25153374, 321.95705521, 324.66257669, 327.36809816,
       330.07361963, 332.7791411 , 335.48466258, 338.19018405,
       340.89570552, 343.60122699, 346.30674847, 349.01226994,
       351.71779141, 354.42331288, 357.12883436, 359.83435583,
       362.5398773 , 365.24539877, 367.95092025, 370.65644172,
       373.36196319, 376.06748466, 378.77300613, 381.47852761,
       384.18404908, 386.88957055, 389.59509202, 392.30

In [30]:
mappings.find_freq_list(a.first_zoom[0].y_axis, 800)

[0, 49]

In [32]:
a.first_zoom[0].y_axis

81

In [17]:
Audio(sound_file, autoplay=True)

NameError: name 'Audio' is not defined

In [20]:
file_name = '../../data/MIDI-Unprocessed_R1_D1-1-8_mid--AUDIO-from_mp3_03_R1_2015_wav--2.wav'
y, sr = librosa.load(file_name, sr=44100)
y = y[len(y)//2:len(y)//2+30*sr]

In [21]:
for i in range(len(res_window)):
#     #STFT
#     res = res_window[i]
#     print(res)
#     result_stft = %timeit -n 3 -r 3 -o librosa.stft(y, n_fft=res)
#     #CQT
#     bpo = hz_to_binperoct(res_hz[i])
#     result_cqt = %timeit -n 3 -r 3 -o librosa.cqt(y, sr=sr, bins_per_octave=bpo, fmin=20, n_bins=10*bpo)
    # LUKIN-TODD, SLS, SWGM
    max_window = res_window[i]
    window_lengths = [int(max_window/8), int(max_window/2), max_window]
    kernel_anal = calc_kernel_size(window_lengths)
    kernel_energy = calc_kernel_size(window_lengths, energy=True)
    result_LT = %timeit -n 1 -r 1 -o LT(y, window_lengths, kernel_anal)
#     result_SWGM = %timeit -n 3 -r 3 -o swgm_time(y, window_lengths)
    result_SLS = %timeit -n 1 -r 1 -o SLS_time(y, window_lengths, kernel_anal, kernel_energy)
#     # OUR SOLUTION
#     res = np.max([int(res_window[i] // 2048), 1])
#     result_OURS = %timeit -n 3 -r 3 -o our_solution(y, res)
    
    result_file = 'results_costly' + str(res_window[i]) + '.csv'

    with open(result_file, 'a') as csvfile:
        writer = csv.writer(csvfile, delimiter=';')
#         writer.writerow([file_name, 'STFT', str(result_stft)])
#         writer.writerow([file_name, 'CQT', str(result_cqt)])
        writer.writerow([file_name, 'LT', str(result_LT)])
#         writer.writerow([file_name, 'SWGM', str(result_SWGM)])
        writer.writerow([file_name, 'SLS', str(result_SLS)])
#         writer.writerow([file_name, 'OURS', str(result_OURS)])

1min 22s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
2min 33s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
2min 59s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
5min 25s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
6min 15s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
11min 10s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
12min 46s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
22min 41s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
25min 40s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
45min 43s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
51min 13s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
1h 32min 47s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


KeyboardInterrupt: 

In [ ]:
Audio(sound_file, autoplay=True)